In [2]:
#import library
import numpy as np
import pandas as pd

In [4]:
#import data

df = pd.read_csv("C://Users/haikalef/Downloads/archive2/nyc-rolling-sales.csv")
df.head()

(84548, 22)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [91]:
df.isnull().sum()

Unnamed: 0                        0
BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BLOCK                             0
LOT                               0
EASE-MENT                         0
BUILDING CLASS AT PRESENT         0
ADDRESS                           0
APARTMENT NUMBER                  0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64

In [92]:
'''
Pertanyaan 1:
1.1 apakah besar tanah dari sebuah rumah mempengaruhi harga jual rumah tersebut?
1.2 berapakah confidence interval 95% dari korelasi pada no 1.1

Pertanyaan 2:
2.1 apakah kolom sale price terdistribusi secara normal?

Pertanyaan 3:
apakah penjualan commercial units lebih besar dari residential units?
'''

'\nPertanyaan 1:\n1.1 apakah besar tanah dari sebuah rumah mempengaruhi harga jual rumah tersebut?\n1.2 berapakah confidence interval 95% dari korelasi pada no 1.1\n\nPertanyaan 2:\n2.1 apakah kolom sale price terdistribusi secara normal?\n\nPertanyaan 3:\napakah penjualan commercial units lebih besar dari residential units?\n'

PERTANYAAN 1.1

Preprocessing kolom Land Square Feet dan Sale Price, cari korelasi nya!

In [93]:
df['SALE PRICE'][5]

' -  '

In [94]:
len(df[df['SALE PRICE'] == df['SALE PRICE'][5]])

14561

In [95]:
median_price = df[df['SALE PRICE'] != df['SALE PRICE'][5]]['SALE PRICE'].median()
median_price

530000.0

In [96]:
df['SALE PRICE'] = df['SALE PRICE'].replace(' -  ', median_price)

In [97]:
len(df[df['SALE PRICE'] == ' -  ']) #cek apakah masih ada strip atau tidak

0

In [98]:
#now check Land Square Feet column, is there any missing value or no? (like strip or null)
not_number = []

for i in range(0, len(df['LAND SQUARE FEET'])):
    temp_numbers = df['LAND SQUARE FEET'][i]
    if temp_numbers.isdigit() == True:
        continue
    else:
        not_number.append(temp_numbers)               

In [99]:
a = np.array(not_number)
np.unique(a)

array([' -  '], dtype='<U4')

In [100]:
median_land = df[df['LAND SQUARE FEET'] != ' -  ']['LAND SQUARE FEET'].median()
median_land

2325.0

In [101]:
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace(' -  ', median_land)

In [102]:
len(df[df['LAND SQUARE FEET'] == ' -  ']) #cek apakah masih ada strip atau tidak

0

In [103]:
df[['LAND SQUARE FEET', 'SALE PRICE']]

,LAND SQUARE FEET,SALE PRICE
0,1633,6625000
1,4616,530000.0
2,2212,530000.0
3,2272,3936272
4,2369,8000000
...,...,...
84543,2400,450000
84544,2498,550000
84545,4000,460000
84546,208033,11693337


In [104]:
print(df['LAND SQUARE FEET'].dtypes)
print(df['SALE PRICE'].dtypes)

object
object


In [105]:
#use pearson's correlation for check the correlation
#h0 = besar tanah tidak mempengaruhi harga jual
#h1 = besar tanah mempengaruhi harga jual

df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].astype(float) #change dtype to float
df['SALE PRICE'] = df['SALE PRICE'].astype(float)

from scipy.stats import pearsonr
data1 = df['LAND SQUARE FEET']
data2 = df['SALE PRICE']
stat, p= pearsonr(data1, data2)

print(f"stat={stat}, p={p}")
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

stat=0.05131154684817121, p=2.1171022715410064e-50
Probably dependent


Karena p-value (2.1171022715410064e-50) dan kurang dari sama dengan alpha (0.05), maka kita dapat reject the null hypothesis, bahwa besar tanah mempengaruhi harga jual.

PERTANYAAN 1.2

Confidence Interval 95% dari korelasi pada no 1.1

In [106]:
from scipy import stats

def pearsonr_ci(x, y, alpha=0.05):
    stat, p = pearsonr(x, y)
    r_z = np.arctanh(stat)
    se = 1/np.sqrt(len(x)-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return lo, hi

pearsonr_ci(data1, data2)

(0.044586383559312597, 0.058032059683572196)

Pada 95% confidence interval memiliki nilai lower bound = 0.044586383559312597, dan upper bound = 0.058032059683572196. Sehingga 95% korelasi tersebar pada range tersebut

PERTANYAAN 2

Apakah kolom gross square feet terdistribusi secara normal?

Dalam kasus ini kita dapat menggunakan Shapiro-Wilk Test untuk mengecek apakah data memiliki Gaussian Distribution 

In [107]:
df['GROSS SQUARE FEET'].dtypes

dtype('O')

In [108]:
#now check Land Square Feet column, is there any missing value or no? (like strip or null)
not_number1 = []

for i in range(0, len(df['GROSS SQUARE FEET'])):
    temp_numbers1 = df['GROSS SQUARE FEET'][i]
    if temp_numbers1.isdigit() == True:
        continue
    else:
        not_number1.append(temp_numbers1)         

In [109]:
b = np.array(not_number1)
b

array([' -  ', ' -  ', ' -  ', ..., ' -  ', ' -  ', ' -  '], dtype='<U4')

In [110]:
print(len(b))
print(np.unique(b))

27612
[' -  ']


In [111]:
#karena ada 27612 missing value " - ", maka dapat menggantikan nilai " - " dengan nilai median
median_gross_land = df[df['GROSS SQUARE FEET'] != ' -  ']['GROSS SQUARE FEET'].median()
median_gross_land

1680.0

In [112]:
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace(' -  ', median_gross_land)

In [113]:
len(df[df['GROSS SQUARE FEET'] == ' -  ']) #cek apakah masih ada strip atau tidak

0

In [115]:
df['GROSS SQUARE FEET']

0         6440
1        18690
2         7803
3         6794
4         4615
         ...  
84543     2575
84544     2377
84545     1496
84546    64117
84547     2400
Name: GROSS SQUARE FEET, Length: 84548, dtype: object

In [116]:
#Shapiro-Wilk Normallity Test

from scipy.stats import shapiro

df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].astype(float) #change dtype to float

data3 = df['GROSS SQUARE FEET']
stat, p = shapiro(data3)
print(f"stat={stat}, p={p}")
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

stat=0.0368003249168396, p=0.0
Probably not Gaussian


C:\Users\haikalef\anaconda3\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Berdasarkan Shapiro-Wilk normallity test, p-value nya tidak lebih dari 0.05, yaitu 0. Maka kolom Gross Square Feet probably not Gaussian, sehingga dapat disimpulkan tidak terdistribusi secara normal

PERTANYAAN 3

Penjualan commercial units = residential units? Olah dua kolom tersebut, lalu kita dapat melihat insight nya

In [117]:
df[['COMMERCIAL UNITS', 'RESIDENTIAL UNITS']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   COMMERCIAL UNITS   84548 non-null  int64
 1   RESIDENTIAL UNITS  84548 non-null  int64
dtypes: int64(2)
memory usage: 1.3 MB


In [118]:
#cek apakah ada missing value seperti " - " pada 2 kolom tersebut
#kolom commercial
not_number2 = []

for i in range(0, len(df['COMMERCIAL UNITS'])):
    temp_numbers2 = df['COMMERCIAL UNITS'][i]
    if temp_numbers2.isdigit() == True:
        continue
    else:
        not_number2.append(temp_numbers2)   

AttributeError: 'numpy.int64' object has no attribute 'isdigit'

data pada kolom commercial merupakan nilai integer, maka dapat dipastikan tidak ada missing value

In [119]:
#kemudian cek pada kolom residential

not_number3 = []

for i in range(0, len(df['RESIDENTIAL UNITS'])):
    temp_numbers3 = df['RESIDENTIAL UNITS'][i]
    if temp_numbers3.isdigit() == True:
        continue
    else:
        not_number3.append(temp_numbers3)   

AttributeError: 'numpy.int64' object has no attribute 'isdigit'

sama seperti pada kolom commercial, kolom residential juga bernilai integer dan dipastikan tidak ada missing value

In [123]:
#h0 = rata" commercial units = rata" residential units
#h1 = rata" commercial units != rata" residential units
#using student's t-test

from scipy.stats import ttest_ind

data4 = df['COMMERCIAL UNITS']
data5 = df['RESIDENTIAL UNITS']
stat, p = ttest_ind(data4, data5)
print(f"stat={stat}, p={p}")
if p > 0.05:
    print('Fail to reject h0')
else:
    print('Reject h0')

stat=-28.247478774335146, p=3.915820421026095e-175
Reject h0


Berarti, dari hasil diatas dapat diketahui bahwa p-value nya sangat kecil, maka kondisi ini menolak/reject hipotesis null (h0). Maka dari itu rata - rata commercial units tidak sama dengan rata-rata residential units

Pada assignment ini, kita harus mencari nilai mean, median, modus, range, variance, standard deviation dari kolom/data spesifik

In [122]:
#mean dari kolom sale price
print('Mean Sale Price: ', df['SALE PRICE'].mean())

#variance dari kolom sale price
print('Variance Sale Price: ', df['SALE PRICE'].var(ddof=1))

#standard deviation dari kolom sale price
print('SD Sale Price: ', df['SALE PRICE'].std(ddof=1))
print()

#median dari kolom land square feet
print('Median LAND SQUARE FEET: ', df['LAND SQUARE FEET'].median())

#modus dari kolom gross square feet
print('Modus Gross Square Feet: ', df['GROSS SQUARE FEET'].mode())
print()

#range dari kolom total units
print('Range Total Units: ', np.ptp(df['TOTAL UNITS']))

Mean Sale Price:  1147900.4933174055
Variance Sale Price:  107756447924177.75
SD Sale Price:  10380580.326945972

Median LAND SQUARE FEET:  2325.0
Modus Gross Square Feet:  0    1680.0
Name: GROSS SQUARE FEET, dtype: float64

Range Total Units:  2261
